# Bài tập nhóm Chương 2

# ------------------------
# 1. Đọc dữ liệu 
# ------------------------

In [15]:
import pandas as pd
import numpy as np
import os
import re
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [16]:


# 1: Đọc dữ liệu từ file CSV
csv_path = "bds_crawl.csv"
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"ERROR: File {csv_path} không tồn tại!")

df = pd.read_csv(csv_path)
print(f"✓ Đã đọc file {csv_path} thành công.")

# Kiểm tra các cột bắt buộc
required_columns = ['Khoảng giá', 'Diện tích', 'Số phòng ngủ', 'Số phòng tắm/vệ sinh', 'Nội thất', 'Pháp lý']
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    raise ValueError(f"ERROR: Thiếu các cột bắt buộc: {', '.join(missing_cols)}")

print("✓ Đã kiểm tra đầy đủ các cột bắt buộc.")

✓ Đã đọc file bds_crawl.csv thành công.
✓ Đã kiểm tra đầy đủ các cột bắt buộc.


In [17]:
print("Thông tin cơ bản về dữ liệu:")
print(f"Kích thước dữ liệu: {df.shape}")
print(f"Số lượng: {len(df)}")
print("\nCác cột trong dữ liệu:")
print(df.columns.tolist())

# Hiển thị 5 dòng đầu tiên của dữ liệu
print("\n5 dòng đầu tiên của dữ liệu:")
display(df.head())


Thông tin cơ bản về dữ liệu:
Kích thước dữ liệu: (2016, 9)
Số lượng: 2016

Các cột trong dữ liệu:
['Link', 'Ngày đăng', 'Loại hình căn hộ', 'Khoảng giá', 'Diện tích', 'Số phòng ngủ', 'Số phòng tắm/vệ sinh', 'Pháp lý', 'Nội thất']

5 dòng đầu tiên của dữ liệu:


,Link,Ngày đăng,Loại hình căn hộ,Khoảng giá,Diện tích,Số phòng ngủ,Số phòng tắm/vệ sinh,Pháp lý,Nội thất
0,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,08/10/2025,Căn hộ chung cư tại Vinhomes Ocean Park Gia Lâm,"3,9 tỷ",58 m²,1 phòng,1 phòng,Hợp đồng mua bán,Cơ bản
1,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,04/10/2025,Căn hộ chung cư tại Vinhomes Ocean Park Gia Lâm,"3,1 tỷ",48 m²,1 phòng,1 phòng,Sổ đỏ/ Sổ hồng,Đầy đủ
2,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,08/10/2025,Căn hộ chung cư tại Legacy Hinoiri,"5,7 tỷ",110 m²,3 phòng,3 phòng,Sổ đỏ/ Sổ hồng,Đầy đủ
3,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,03/10/2025,Căn hộ chung cư tại Chung cư The Wisteria,"5,6 tỷ",73 m²,2 phòng,2 phòng,Hợp đồng mua bán,Nội thất bàn giao hiện đại
4,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,09/10/2025,Căn hộ chung cư tại Chung cư The Wisteria,"6,7 tỷ",83 m²,2 phòng,2 phòng,Hợp đồng mua bán,Cơ bản


# ----------------------------------------------------------------------
# 2. LÀM SẠCH ĐỊNH DẠNG DỮ LIỆU THÔ (FEATURE ENGINEERING CƠ BẢN)
# ----------------------------------------------------------------------


In [18]:

# 2.1. Làm sạch cột Khoảng giá
# Sử dụng đơn vị VNĐ làm đơn vị gốc để tránh nhầm lẫn


def _normalize_number_string(num_s: str) -> str:
    """Chuẩn hoá chuỗi số: xử lý dấu phân cách hàng nghìn và dấu thập phân.
    Quy tắc đơn giản:
      - Nếu chứa cả '.' và ',' giả định '.' là hàng nghìn, ',' là thập phân -> remove '.' and replace ','->'.'
      - Nếu chỉ chứa ',' -> treat ',' as decimal -> replace ','->'.'
      - Nếu chỉ chứa '.' -> keep '.' (treat as decimal)  
    """
    if '.' in num_s and ',' in num_s:
        return num_s.replace('.', '').replace(',', '.')
    if ',' in num_s:
        return num_s.replace(',', '.')
    return num_s


def clean_price_to_vnd(price):
    s = str(price)
    if pd.isna(price) or 'thỏa thuận' in s.lower():
        return np.nan

    s_low = s.lower().strip()
    # tìm chuỗi số đầu tiên (có thể chứa '.' và ',')
    m = re.search(r"[\d.,]+", s_low)
    if not m:
        return np.nan
    num_s = m.group(0)
    num_s = _normalize_number_string(num_s)

    try:
        value = float(num_s)
    except ValueError:
        return np.nan

    # Quy đổi theo đơn vị xuất hiện trong chuỗi gốc
    if 'tỷ' in s_low:
        return value * 10**9
    if 'triệu/m²' in s_low:
        # Bỏ qua các giá trên m2 (xử lý sau nếu cần)
        return np.nan
    if 'triệu' in s_low:
        return value * 10**6
    if 'vnđ' in s_low or 'vnd' in s_low:
        return value

    # Trường hợp không ghi đơn vị rõ ràng: duy trì hành vi cũ nhưng thêm ngưỡng an toàn
    # Nếu giá nhỏ (<1000) giả định là số theo đơn vị tỷ (ví dụ: '40' -> 40 tỷ),
    # nếu số lớn (>=1000) có thể đã là VNĐ thô (ví dụ 3900000000) -> trả về như VNĐ
    if value < 1000:
        return value * 10**9
    return value

# Áp dụng hàm mới, tạo cột giá trị tuyệt đối (VNĐ)
df['Gia_VND'] = df['Khoảng giá'].apply(clean_price_to_vnd)


In [19]:
# 2.2. Làm sạch cột Diện tích
def clean_area_improved(area):
    if pd.isna(area):
        return np.nan

    s = str(area).lower().strip()
    # loại bỏ ký tự không phải số, '.', ','
    m = re.search(r"[\d.,]+", s)
    if not m:
        return np.nan
    num_s = m.group(0)

    # cùng chuẩn hoá số như trên
    if '.' in num_s and ',' in num_s:
        num_s = num_s.replace('.', '').replace(',', '.')
    elif ',' in num_s:
        num_s = num_s.replace(',', '.')

    try:
        return float(num_s)
    except ValueError:
        return np.nan

# Lưu cột diện tích đã làm sạch
df['Diện tích (m²) đã làm sạch'] = df['Diện tích'].apply(clean_area_improved)


In [20]:

# 2.3. Làm sạch Số phòng ngủ và Số phòng tắm/vệ sinh
def clean_room(room):
    if pd.isna(room):
        return np.nan
    room_str = str(room).lower().replace(' phòng', '').strip()
    try:
        return int(room_str)
    except ValueError:
        return np.nan

# Điền giá trị thiếu (NaN) bằng 0 cho các cột phòng
df['Số phòng ngủ'] = df['Số phòng ngủ'].apply(clean_room).fillna(0).astype(int)
df['Số phòng tắm/vệ sinh'] = df['Số phòng tắm/vệ sinh'].apply(clean_room).fillna(0).astype(int)
df = df.drop(columns = ['Khoảng giá', 'Diện tích'])



# -----------------------------------------------
# 3. XỬ LÝ GIÁ TRỊ THIẾU
# -----------------------------------------------

In [21]:

# 3.1. Điền giá trị thiếu cho Gia_VND bằng Trung vị (Median)
nan_count_gia = df['Gia_VND'].isna().sum()
median_gia = df['Gia_VND'].median()
df['Gia_VND'] = df['Gia_VND'].fillna(median_gia)
print(f"✅ Đã điền {nan_count_gia} giá trị thiếu trong cột 'Gia_VND' bằng Trung vị ({median_gia:,.0f} VNĐ).")

# 3.2. Điền giá trị thiếu cho Nội thất bằng 'Chưa rõ'
nan_count_nt = df['Nội thất'].isna().sum()
df['Nội thất'] = df['Nội thất'].fillna('Chưa rõ')
print(f"✅ Đã điền {nan_count_nt} giá trị thiếu trong cột 'Nội thất' bằng 'Chưa rõ'.")

print("\nTổng quan dữ liệu trước khi xử lý ngoại lai:")
print(df[['Gia_VND', 'Diện tích (m²) đã làm sạch']].describe())

✅ Đã điền 521 giá trị thiếu trong cột 'Gia_VND' bằng Trung vị (11,500,000,000 VNĐ).
✅ Đã điền 930 giá trị thiếu trong cột 'Nội thất' bằng 'Chưa rõ'.

Tổng quan dữ liệu trước khi xử lý ngoại lai:
            Gia_VND  Diện tích (m²) đã làm sạch
count  2.016000e+03                 2015.000000
mean   2.049204e+10                  105.183859
std    4.455665e+10                   85.096757
min    2.000000e+07                    1.000000
25%    8.000000e+09                   57.000000
50%    1.150000e+10                   82.500000
75%    1.850000e+10                  120.000000
max    9.990000e+11                  999.700000


# ----------------------------------------------------------------------
# 3.3. NHÓM GIÁ TRỊ PHÂN LOẠI KHÔNG NHẤT QUÁN
# ----------------------------------------------------------------------


In [22]:

# 3.3.1. Nhóm cột Nội thất
def group_noi_that(text):
    text = str(text).lower()
    if 'full' in text or 'đầy đủ' in text or 'cao cấp' in text:
        return 'Day du/Cao cap'
    if 'cơ bản' in text or 'liền tường' in text:
        return 'Co ban'
    if 'thô' in text or 'hoàn thiện mặt ngoài' in text:
        return 'Ban giao tho'
    return 'Chua ro'

df['Nội thất'] = df['Nội thất'].apply(group_noi_that)

# 3.3.2. Nhóm cột Pháp lý
def group_phap_ly(text):
    text = str(text).lower()
    if 'sổ đỏ' in text or 'sổ hồng' in text:
        return 'So do/So hong'
    if 'hợp đồng mua bán' in text:
        return 'HDMB'
    return 'Khac/Chua co'

df['Pháp lý'] = df['Pháp lý'].apply(group_phap_ly)

print("✅ Đã nhóm các giá trị Nội thất và Pháp lý. Số lượng giá trị độc nhất đã giảm.")

✅ Đã nhóm các giá trị Nội thất và Pháp lý. Số lượng giá trị độc nhất đã giảm.


# ----------------------------------------------------------------------
# 4. Xử lý Ngoại lai (OUTLIERS) (ĐÃ THÊM LỚP BẢO VỆ CHỐNG NaN)
# ----------------------------------------------------------------------

In [23]:


outlier_cols = ['Gia_VND', 'Diện tích (m²) đã làm sạch', 'Số phòng ngủ', 'Số phòng tắm/vệ sinh']

print("\n--- 4: Xử lý Giá trị Ngoại lai (Outliers) bằng IQR Capping ---")
for col in outlier_cols:
    # Bổ sung: Đảm bảo cột không còn NaN bằng cách điền median của chính cột đó
    # Điều này chống lại lỗi workflow và đảm bảo np.where() hoạt động an toàn
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_count = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
    
    # Xử lý ngoại lai bằng cách thay thế (Winsorizing)
    df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
    
    if outliers_count > 0:
        print(f"✅ Đã xử lý {outliers_count} ngoại lai trong cột '{col}'.")
    else:
         print(f"➖ Cột '{col}' không có ngoại lai để xử lý.")


--- 4: Xử lý Giá trị Ngoại lai (Outliers) bằng IQR Capping ---
✅ Đã xử lý 242 ngoại lai trong cột 'Gia_VND'.
✅ Đã xử lý 155 ngoại lai trong cột 'Diện tích (m²) đã làm sạch'.
✅ Đã xử lý 51 ngoại lai trong cột 'Số phòng ngủ'.
✅ Đã xử lý 93 ngoại lai trong cột 'Số phòng tắm/vệ sinh'.


In [24]:
df.to_csv('bds_clean_grouped.csv', index=False, encoding='utf-8-sig')

print("✅ Đã lưu file 'bds_clean_grouped.csv' thành công.")

✅ Đã lưu file 'bds_clean_grouped.csv' thành công.


# -------------------------------------------
# 5. Co giãn Min-Max (Normalization)
# -------------------------------------------

In [25]:

cols_to_scale = ['Gia_VND', 'Diện tích (m²) đã làm sạch', 'Số phòng ngủ', 'Số phòng tắm/vệ sinh']

# Kiểm tra sự tồn tại của các cột
available_cols = [col for col in cols_to_scale if col in df.columns]
missing_cols = [col for col in cols_to_scale if col not in df.columns]
if missing_cols:
    print(f"⚠️ Cảnh báo: Không tìm thấy các cột: {', '.join(missing_cols)}")
if not available_cols:
    raise ValueError("Không có cột nào để scale!")

# Co giãn Min-Max (Normalization)
scaler_minmax = MinMaxScaler()
df_normalized = df.copy() 
df_normalized[available_cols] = scaler_minmax.fit_transform(df_normalized[available_cols])

print("\n--- 5: Co giãn Min-Max đã hoàn tất ---")
print("Tóm tắt thống kê trước và sau khi scale:")
print("\nTRƯỚC KHI SCALE:")
print(df[available_cols].describe())



--- 5: Co giãn Min-Max đã hoàn tất ---
Tóm tắt thống kê trước và sau khi scale:

TRƯỚC KHI SCALE:
            Gia_VND  Diện tích (m²) đã làm sạch  Số phòng ngủ  \
count  2.016000e+03                 2016.000000   2016.000000   
mean   1.446219e+10                   96.550187      2.319444   
std    9.622235e+09                   53.412485      2.506397   
min    2.000000e+07                    1.000000      0.000000   
25%    8.000000e+09                   57.000000      0.000000   
50%    1.150000e+10                   82.500000      2.000000   
75%    1.850000e+10                  120.000000      4.000000   
max    3.425000e+10                  214.500000     10.000000   

       Số phòng tắm/vệ sinh  
count           2016.000000  
mean               1.905010  
std                2.179769  
min                0.000000  
25%                0.000000  
50%                2.000000  
75%                3.000000  
max                7.500000  


In [26]:
print("\nSAU KHI SCALE (Phạm vi [0, 1]):")
print(df_normalized[available_cols].describe())


SAU KHI SCALE (Phạm vi [0, 1]):
           Gia_VND  Diện tích (m²) đã làm sạch  Số phòng ngủ  \
count  2016.000000                 2016.000000   2016.000000   
mean      0.421916                    0.447542      0.231944   
std       0.281105                    0.250176      0.250640   
min       0.000000                    0.000000      0.000000   
25%       0.233129                    0.262295      0.000000   
50%       0.335378                    0.381733      0.200000   
75%       0.539877                    0.557377      0.400000   
max       1.000000                    1.000000      1.000000   

       Số phòng tắm/vệ sinh  
count           2016.000000  
mean               0.254001  
std                0.290636  
min                0.000000  
25%                0.000000  
50%                0.266667  
75%                0.400000  
max                1.000000  


#  ------------------------------------------------
# 6. Mã hóa các biến phân loại
# -------------------------------------------------

In [27]:

cols_to_encode = ['Nội thất', 'Pháp lý']

# pd.get_dummies thực hiện One-Hot Encoding
df_final = pd.get_dummies(df_normalized, columns=cols_to_encode, prefix=['Noi_that', 'Phap_ly'], drop_first=True)

print("--- 6: Bảng Tóm tắt Cột đã Mã hóa (One-Hot Encoding) ---")
display(df_final.filter(regex='Noi_that|Phap_ly').head())
print(f"Tổng số cột sau khi mã hóa đã giảm xuống còn: {df_final.shape[1]} cột.")

--- 6: Bảng Tóm tắt Cột đã Mã hóa (One-Hot Encoding) ---


,Noi_that_Chua ro,Noi_that_Co ban,Noi_that_Day du/Cao cap,Phap_ly_Khac/Chua co,Phap_ly_So do/So hong
0,False,True,False,False,False
1,False,False,True,False,True
2,False,False,True,False,True
3,True,False,False,False,False
4,False,True,False,False,False


Tổng số cột sau khi mã hóa đã giảm xuống còn: 12 cột.


In [28]:
# Lưu lại file đã được làm sạch và chuẩn hóa
df_final.to_csv('bds_clean_scaled_encoded.csv', index=False, encoding='utf-8-sig')

print("\n✅ [HOÀN TẤT] Dữ liệu đã được chuẩn bị đầy đủ và sẵn sàng cho Mô hình Học máy.")


✅ [HOÀN TẤT] Dữ liệu đã được chuẩn bị đầy đủ và sẵn sàng cho Mô hình Học máy.
